---
title : "Real-time of what-if overstromingsrisico's"
---

Risico's worden berekend door kansen te combineren met gevolgen. In de toolbox Continu Inzicht worden fragilitycurves gebruikt om de conditionele kans op falen van een sectie (per faalmechanisme) te bepalen. De gevolgen van een overstroming worden bepaald met behulp van overstromingsscenarioberekeningen, ook wel aangeduid met scenario's. Uit een overstroomd gebied worden gevolgen afgeleid zoals economische schade, slachtoffers, en getroffenen. <br>
Door conditionele kansen met gevolgen te combineren kunnen conditionele risico's worden bepaald, bestaande uit de verwachte economische schade van een gebied, het verwachte aantal slachtoffers en/of getroffen in een gebied en/of de plaatsgebonden overstromingskans van een locatie. <br>

Deze risico's kunnen helpen om prioriteiten te stellen tijdens een hoogewatern, waarbij er niet alleen een kans of een afzonderlijk gevolg wordt beschouwd. <br>

Om de scenarios te kunnen combineren moeten er een aantal stappen worden doorlopen:
- Omrekenen van dijkvakkansen naar trajectdeelkansen ([CalculateFloodScenarioProbability](#sec-CalculateFloodScenarioProbability))
- Belasting per deeltraject bepalen ([LoadFromFloodScenarioProbabilit](#sec-LoadFromFloodScenarioProbabilit))
- Met de belasting een bijpassende overstromingscenario selecteren ([SelectFloodScenarioFromLoad ](#sec-SelectFloodScenarioFromLoad))
- Van het scenario en de curve een geagregeerd risico bepalen ([CalculateFloodRisk](#sec-CalculateFloodRisk))


In [ ]:
# initialiseer de (toolbox continu inzicht) modules
from pathlib import Path

from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter

## CalculateFloodScenarioProbability{sec-CalculateFloodScenarioProbability}

Dit voorbeeld laat zien hoe met behulp van de `CalculateFloodScenarioProbability` scenariokansen berekend worden uit kansen per sectie en per faalmachanisme. <br>
Eerst worden de kansen per sectie en faalmachanisme ingeladen, daarna worden deze gecombineerd naar een kans over alle secties (voorheen aangeduid als de ringkans) per faalmechanisme. Daarna worden de kansen over alle secties per faalmechanisme gecombineerd naar 1 kans over alle secties en over alle faalmechanismen. <br>
Deze volgorde (eerst combineren over alle sectie en daarna over alle faalmechanismen) is bewust gekozen omdat per faalmechanisme verschillende met andere correlaties wordt gerekend. Voor het faalmechanisme GEKB wordt verondersteld dat alle secties volledig afhankelijk falen en voor de overige faalmechanismen wordt verondersteld dat de secties volledig onafhankelijk falsen. Door gekozen volgorde bij het combineren van faalkansen aan te houden, kan dit onderscheid geborgd worden. Tussen de verschillende faalmechanismen wordt volledige onafhankelijkheid verondersteld. N.B. in een volledige probabilistische berekening (waarbij niet met fragility curves wordt gerekend) kan de daadwerkelijke correlatie worden berekend en is het ook mogelijk om te rekenen met gedeeltelijke afhankelijkheid. Dat is in een werkwijze met fragilitycurves niet mogelijk. <br>

```yaml|
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [-24,0,24,36]

DataAdapter:
    default_options:
        csv:
            sep: ","
    input_failuremechanism:
        type: csv
        file: "rekentest_failuremechanism.csv"
    input_probabilities_failuremechanisme_sections:
        type: csv
        file: "rekentest_probabilities_failuremechanisme_sections.csv"
    input_sections_in_segment:
        type: csv
        file: "rekentest_sections_in_segment.csv"
    output_scenario_failure_prob_segments:
        type: csv
        index: false
        file: "hidden_rekentest_scenario_failure_prob_segments.csv"
    output_combined_failure_prob_all_sections:
        type: csv
        index: false
        file: "hidden_rekentest_combined_failure_prob_all_sections.csv"
```

#TODO
- loop over momenten inbouwen
- nog een lengte-effect factor verwerken in de bepalen van de faalkans per segment

In [ ]:
# laadt de configuratie en initialiseer de data adapter
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(
    config_path=path / "rekentest_calculate_flood_scenario_probability.yaml"
)
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
# druk een overzicht van de faalmechanismen af
data_adapter.input("input_failuremechanism")

In [ ]:
# druk een overzicht van de secties per segment af
data_adapter.input("input_sections_in_segment")

In [ ]:
# druk een overzicht van de faalkansen per sectie en faalmechanisme af
data_adapter.input("input_probabilities_failuremechanisme_sections")

In [ ]:
# importeer en initialiseer de module voor het berekenen van de scenariokansen per segment
from toolbox_continu_inzicht.flood_scenarios import CalculateFloodScenarioProbability

In [ ]:
# maak een instantie van de klasse CalculateFloodScenarioProbability
calculate_flood_scenario_probability = CalculateFloodScenarioProbability(
    data_adapter=data_adapter
)

In [ ]:
# voer de berekening van de scenariokansen uit
calculate_flood_scenario_probability.run(
    input=[
        "input_failuremechanism",
        "input_probabilities_failuremechanisme_sections",
        "input_sections_in_segment",
    ],
    output=[
        "output_scenario_failure_prob_segments",
        "output_combined_failure_prob_all_sections",
    ],
)

In [ ]:
# druk een overzicht van de scenariokansen faalkansen per segment en over alle faalmechanismen af
calculate_flood_scenario_probability.df_out_scenario_failure_prob_segments

In [ ]:
# druk een overzicht van de gecombineerde faalkansen over alle secties en over alle faalmechanismen af
calculate_flood_scenario_probability.df_out_combined_failure_prob_all_sections

## LoadFromFloodScenarioProbability

Voor het bepalen van de risico van een overstroming is het nodig om een bijpassend overstromingsscenario te selecteren. Er zijn meerdere overstromingsscenario's beschikbaar die verschillende hydraulische belastingen representeren. <br>
Afhankelijk van de scenariokans van een deeltraject (segment) kan een bijpassend overstromingsscenario worden geselecteerd. Hiertoe is het nodig om de hydraulische belasting per deeltraject te bepalen. Dat kan met behulp van de fragilitycurves. Hiertoe is het noodzakelijk om 1 representatieve fragilitycurve per deeltraject te selecteren. Dat doen we door voor elk deeltraject (segment) 1 sectie te selecteren. De fragilitycurve van deze sectie wordt dan gebruikt om de scenariekans van het deeltraject (segement) om te zetten in een hydraulische belasting, die daarna gebruik kan worden om een bijpassend overstromingsscenario te selecteren voor het deeltraject (segment). <br>

```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 36 ]
    LoadFromFloodScenarioProbability:
        failuremechanism_id_combined: 1

DataAdapter:
    default_options:
        csv:
            sep: ","
    input_scenario_failure_prob_segments:
        type: csv
        file: "hidden_rekentest_scenario_failure_prob_segments.csv"
    input_section_to_segment:
        type: csv
        file: "rekentest_section_to_segment.csv"
    input_section_fragility_curves:
        type: csv
        file: "rekentest_fragilitycurves.csv"
    output_scenario_loads:
        type: csv
        path: "hidden_rekentest_scenario_loads.csv"

```

Dit is specifiek voor een moment, over meerdere momenten loopen moet op een hoger viveau.

#TODO
- ~~loop over momenten inbouwen~~
- ~~bij selectie van fragilitycurves rekening houden met faalmechanisme_id (COMB) en met maatregel_id (actieve maatregel)!~~
    > Tests nog nalopen

In [ ]:
# laadt de configuratie en initialiseer de data adapter
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(
    config_path=path / "rekentest_load_from_flood_scenario_probability.yaml"
)
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
# druk een overzicht van de scenariokansen per segment af
data_adapter.input("input_scenario_failure_prob_segments")

In [ ]:
# druk een overzicht van de mapping van sectie naar segment af
data_adapter.input("input_section_to_segment")

In [ ]:
# druk een overzicht van de fragiliteitscurves per faalmechanisme af
data_adapter.input("input_section_fragility_curves")

In [ ]:
# importeer en initialiseer de module voor het bepalen van de belastingen vanuit scenariokansen
from toolbox_continu_inzicht.flood_scenarios import LoadFromFloodScenarioProbability

In [ ]:
# voer de bepaling van de hydraulische belastingen uit
load_from_flood_scenario_probability = LoadFromFloodScenarioProbability(
    data_adapter=data_adapter
)
load_from_flood_scenario_probability.run(
    input=[
        "input_scenario_failure_prob_segments",
        "input_section_to_segment",
        "input_section_fragility_curves",
    ],
    output="output_scenario_loads",
)

In [ ]:
# druk een overzicht van de belastingen overeenkomstig de scenariokansen (per segment) af
load_from_flood_scenario_probability.df_out_scenario_loads

## SelectFloodScenarioFromLoad

Voor het bepalen van de risico van een overstroming is het nodig om een bijpassend overstromingsscenario te selecteren. Er zijn meerdere overstromingsscenario's beschikbaar die verschillende hydraulische belastingen representeren. Per scenario is een hydraulische belasting (per deeltraject) bepaald. Met deze hydraulische belasting kan een bijpassend overstromingsscenario geselcteerd worden voor het deeltraject (segment). <br>

```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 36 ]

DataAdapter:
    default_options:
        csv:
            sep: ","
    input_scenarios_loads:
        type: csv
        path: "hidden_rekentest_scenario_loads.csv"
    input_consequences_loads:
        type: csv
        path: "rekentest_consequences_loads.csv"
    output_scenario_consequences_grids:
        type: csv
        path: "hidden_rekentest_scenario_consequences_grids.csv"
```

In [ ]:
# laadt de configuratie en initialiseer de data adapter
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(config_path=path / "rekentest_select_flood_scenario_from_load.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
# druk een overzicht van de belastingen overeenkomstig de scenariokansen (per segment) af
data_adapter.input("input_scenarios_loads")

In [ ]:
# druk een overzicht van de gevolgen voor verschillende hydraulische belastingniveau's af
data_adapter.input("input_consequences_loads")

In [ ]:
# importeer en initialiseer de module voor het bepalen van de bijbehorende scenario's vanuit belastingen
from toolbox_continu_inzicht.flood_scenarios import SelectFloodScenarioFromLoad

In [ ]:
# voer de bepaling van het bijbehorende gevolggrid uit
select_flood_scenario_from_load = SelectFloodScenarioFromLoad(data_adapter=data_adapter)
select_flood_scenario_from_load.run(
    input=[
        "input_scenarios_loads",
        "input_consequences_loads",
    ],
    output="output_scenario_consequences_grids",
)

In [ ]:
# druk een overzicht van de bijbehorende gevolgen per scenario af
select_flood_scenario_from_load.df_out_scenario_consequences_grids

### Alternatief: keuze uit twee scenarios 

Standaard wordt er per scenariokans, één overstromingsscenario geselecteerd op basis van de hydraulische belasting. Met de parameter `return_two_scenarios` kan ervoor gekozen worden om twee overstromingsscenarios te selecteren. De gebruiker kan hiermee zelf bepalen welk scenario hij wil gebruiken voor de risico berekening. Of eventueel de gemiddelde schade van de twee scenario's gebruiken, of beide tonen. <br>

```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 36 ]
    SelectFloodScenarioFromLoad:
        return_two_scenarios: True

DataAdapter:
    default_options:
        csv:
            sep: ","
    input_scenarios_loads:
        type: csv
        path: "hidden_rekentest_scenario_loads.csv"
    input_consequences_loads:
        type: csv
        path: "rekentest_consequences_loads.csv"
    output_scenario_consequences_grids:
        type: csv
        path: "hidden_rekentest_scenario_consequences_grids.csv"
```

In [ ]:
# laadt de configuratie en initialiseer de data adapter
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(
    config_path=path / "rekentest_select_flood_scenario_from_load_two_scenarios.yaml"
)
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
# druk een overzicht van de belastingen overeenkomstig de scenariokansen (per segment) af
data_adapter.input("input_scenarios_loads")

In [ ]:
# druk een overzicht van de gevolgen voor verschillende hydraulische belastingniveau's af
data_adapter.input("input_consequences_loads")

In [ ]:
# importeer en initialiseer de module voor het bepalen van de bijbehorende scenario's vanuit belastingen
from toolbox_continu_inzicht.flood_scenarios import SelectFloodScenarioFromLoad

In [ ]:
# voer de bepaling van het bijbehorende gevolggrid uit
select_flood_scenario_from_load = SelectFloodScenarioFromLoad(data_adapter=data_adapter)
select_flood_scenario_from_load.run(
    input=[
        "input_scenarios_loads",
        "input_consequences_loads",
    ],
    output="output_scenario_consequences_grids",
)

In [ ]:
# druk een overzicht van de bijbehorende gevolgen per scenario af
# TODO verwijder de index kolom in de CSV output
select_flood_scenario_from_load.df_out_scenario_consequences_grids

## CalculateFloodRisk

Het (conditionele) risico wordt bepaald door de (conditionele) kans te combineren met het gevolg. In dit geval worden de gevolgen van een overstroming gecombineerd met de scenariokans van een deeltraject (segment). <br>
Met behulp van de `CalculateFloodRisk` module worden risico's berekend voor 4 verschillende risicomate: (1) verwachte economische schade, (2) verwacht aantal slachtoffers, (3) verwacht aantal getroffenen, en (4) plaatsgebonden overstromingskans. <br>
In eerste instantie wordt het risico berekend per gridcel. Daarna worden de risico's geaggregeerd naar gebieden die gedefinieerd zijn in een shape-bestand.


```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 36 ]
    CalculateFloodRisk:
        aggregate_methods: # dit zijn de standaard aggregatie-methoden per risicomaat (kunnen worden overschreven in de config)
            casualties: sum
            damage: sum
            flooding: median
            affectedpeople: sum
            waterdepth: sum
        per_hectare: False
        
DataAdapter:
    default_options:
        csv:
            sep: ","
            Index: False
    input_scenario_failure_prob_segments:
        type: csv
        path: "hidden_rekentest_scenario_failure_prob_segments.csv"
    input_scenario_consequences_grids:
        type: csv
        path: "hidden_rekentest_scenario_consequences_grids.csv"
    input_areas_to_aggregate:
        type: shape
        path: "rekentest_areas_to_aggregate.geojson"
    flood_risk_local_file:
        type: flood_risk_local_file
        path: '' # gebruik de data_dir
        scenario_path: "flood_scenarios" # en een map dieper kijk in de floodscenarios dir
        #abs_path_user: "tests/src/flood_scenarios/data_sets/flood_scenarios" # dit kan ook gebruikt worden
    output_flood_risk_results:
        type: csv
        path: "hidden_rekentest_flood_risk_results.csv"

```

In [ ]:
# laadt de configuratie en initialiseer de data adapter
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(config_path=path / "rekentest_calculate_flood_risk.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
# druk een overzicht van de scenariokansen per segment af
data_adapter.input("input_scenario_failure_prob_segments")

In [ ]:
# druk een overzicht van de bijbehorende gevolgen per scenario af
data_adapter.input("input_scenario_consequences_grids")

In [ ]:
# druk de gebieden af waarvoor het overstromingsrisico wordt geaggregeerd
data_adapter.input("input_areas_to_aggregate")

In [ ]:
# importeer en initialiseer de module voor het berekenen van het overstromingsrisico in 4 verschillende formaten
from toolbox_continu_inzicht.flood_scenarios.calculate_flood_risk import (
    CalculateFloodRisk,
)

In [ ]:
# voer de berekening van het risico op grid niveau uit
calculate_flood_risk = CalculateFloodRisk(data_adapter=data_adapter)
calculate_flood_risk.run(
    input=[
        "input_scenario_failure_prob_segments",
        "input_scenario_consequences_grids",
        "input_areas_to_aggregate",
        "flood_risk_local_file",
    ],
    output="output_flood_risk_results",
)

In [ ]:
data_adapter.input("flood_risk_local_file")

In [ ]:
calculate_flood_risk.df_out.head(5)

In [ ]:
import matplotlib.pyplot as plt

gdf_results = calculate_flood_risk.df_out

columns = ["casualties", "damage", "flooding", "affected_people"]
rename = {
    "casualties": "Slachtoffers",
    "damage": "Schade",
    "flooding": "Plaatgebonden overstromingskans",
    "affected_people": "Getroffen mensen",
}
fig, axs = plt.subplots(2, 2, figsize=(7, 5))
ax = axs.flatten()
for index, column in enumerate(columns):
    gdf_results.plot(column=column, ax=ax[index])
    plt.colorbar(ax[index].collections[0], ax=ax[index], orientation="horizontal")
    ax[index].set_title(rename[column])
plt.tight_layout()

Herhaal per ha
```yaml
GlobalVariables:
    ...
    CalculateFloodRisk:
        ...
        per_hectare: True
        columns_per_hectare:
            - casualties
            - damage
            - affectedpeople

```


In [ ]:
config = Config(config_path=path / "rekentest_calculate_flood_risk_per_ha.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)
calculate_flood_risk = CalculateFloodRisk(data_adapter=data_adapter)
calculate_flood_risk.run(
    input=[
        "input_scenario_failure_prob_segments",
        "input_scenario_consequences_grids",
        "input_areas_to_aggregate",
        "flood_risk_local_file",
    ],
    output="output_flood_risk_results",
)

In [ ]:
import matplotlib.pyplot as plt

gdf_results = calculate_flood_risk.df_out

columns = ["casualties_per_ha", "damage_per_ha", "flooding", "affected_people_per_ha"]
rename = {
    "casualties_per_ha": "Slachtoffers per hectare",
    "damage_per_ha": "Schade per hectare",
    "flooding": "Plaatgebonden overstromingskans",
    "affected_people_per_ha": "Getroffen mensen per hectare",
}
fig, axs = plt.subplots(2, 2, figsize=(7, 5))
ax = axs.flatten()
for index, column in enumerate(columns):
    gdf_results.plot(column=column, ax=ax[index])
    plt.colorbar(ax[index].collections[0], ax=ax[index], orientation="horizontal")
    ax[index].set_title(rename[column])
plt.tight_layout()